# Decode VIN for Linked MV104/SPARCS dataset

This script uses the truncated VIN (Vehicle Identification Number) in the linked dataset and NHTSA's VIN API to get vehicle body type information. 

The resulting file is saved as a csv file: *vin_api_data.csv*

In [1]:
import pandas as pd
pd.options.display.max_rows = 500
pd.options.display.max_columns = 130

import numpy as np

import matplotlib.pyplot as plt
% matplotlib inline
plt.style.use('seaborn-poster')
plt.style.use('ggplot')

import databuild as db
import sys
sys.path.insert(0,'/home/deena/Documents/data_munge/ModaCode/')
import moda

/usr/local/lib/python2.7/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# read in DMV data into 3 tables
crash,ind,veh = db.readDMV()

#read in DMV-SPARCS linked data
linked = db.readLinked()


/usr/local/lib/python2.7/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


crash table (522108, 26)
person table (1502797, 22)
vehicle table (1092922, 20)
linked (76763, 131)
linked after dropping no police reports (69657, 131)


In [10]:
# vehicles involved in matched crashes
veh = veh[veh.CS_ID.isin(linked.CS_ID)]
veh.shape

(124954, 20)

In [52]:
# keeping only veh with VINs
veh = veh[pd.notnull(veh.VIN)]
veh = veh[pd.notnull(veh.CV_VEH_YEAR)]

veh.shape

(81879, 20)

In [24]:
# Take a random sample of crashes_df
#crash_sample = veh.sample(n=50)
#crash_sample.head()

# all crashes
crash_sample = veh

,CS_ID,CV_ID,VEHBDYT_ID,REGT_CDE,PACCACTT_ID,TBCT_DMV_CDE,DIRCTT_CDE,FT_ID,CV_VEH_YEAR,CV_REG_STATE_CDE,CV_WEIGHT_LBS,CV_PSGR_NUM,CV_CYLNDR,CV_VEHMAKE_DESCR,CFT_CDE1,CFT_CDE2,SHZMTT_ID,VEH_EVNTT_ID,VIN,DMV_VIN_NUM
854079,34480872,10583739,5.0,16,10,-3,E,1,2010,NY,None,-2,6,FORD,-2,-2,None,-2,2FMDK4JC9AB,None
921770,34656038,10895982,5.0,16,1,-3,N,1,1998,NY,None,2,6,CHEVR,7,-3,None,-3,1GNEL19W7WB,None
843583,34423104,10481594,5.0,16,1,-3,S,1,1997,NY,None,4,6,FORD,4,19,None,-3,1FMDU34E7VZ,None
932755,34654147,10892693,6.0,54,1,-3,N,1,2007,NY,None,1,4,TOYOT,-3,-3,None,-3,4T1BE46K07U,None
941980,34635051,10858577,60.0,19,6,-3,S,2,2000,NY,None,2,8,CHEVR,4,-3,None,-3,1GBHG31F4Y1,None


should have deduped vehicle file for duplicate VINS before running the api

In [58]:
# Read in the vehicle information using the NHTSA vehicle api 
vin_api_data = pd.DataFrame()

nhtsaURL1 = 'https://vpic.nhtsa.dot.gov/api/vehicles/decodevinvalues/'
nhtsaURL2 = '*BA?format=json&modelyear='


for index, row in crash_sample.iterrows():
    vin = row['VIN']
    year = row['CV_VEH_YEAR']
    try:
        vin_data = pd.read_json(nhtsaURL1+vin+nhtsaURL2+year)
        vin_norm = pd.DataFrame(vin_data['Results'][0],index=[0])

        # Add each line of results (individual API call) to vin_api_data 
        vin_api_data = pd.concat([vin_api_data, vin_norm], axis=0)
    except:
        print 'try except error'

print vin_api_data.shape

try except error
try except error
try except error
(81876, 144)


In [ ]:
# The api added three character to the VIN, 
# remove them before joining to original table
vin_api_data['VIN'] = vin_api_data['VIN'].str[:-3]

In [82]:
# remove duplicates, would not be necessary if we did this before running the api
vin_api_data.drop_duplicates(subset='VIN',inplace=True)
vin_api_data.shape

(44586, 144)

In [90]:
#save decoded vin to csv
#vin_api_data.to_csv('vin_api_data.csv',index=False,encoding='utf-8')

In [84]:
# join back to crash table
# Use that 11 digit vin to join with the original vehicle table
veh_merge = veh.merge(vin_api_data,how='left',on='VIN')
veh_merge.shape

(81879, 163)

In [85]:
# the VIN has has two variables that refer to type of vehicle.
# Let's see how they relate to each other.
veh_merge.groupby(['VehicleType','BodyClass'])[['VIN']].count()

VIN
VehicleType                          BodyClass                                                
                                                                                           174
BUS                                                                                         24
                                     Bus                                                   625
                                     Bus - School Bus                                       74
                                     Incomplete                                              1
                                     Incomplete - School Bus Chassis                        20
                                     Truck                                                  80
                                     Van                                                   266
                                     Wagon                                                 490
INCOMPLETE VEHICLE                                                                          18
                                     Cabriolet/Convertible                                   3
                                     Cargo Van                                              46
                                     Crossover Utility Vehicle (CUV)                         2
                                     Glider                                                  9
                                     Incomplete                                             90
                                     Incomplete - Bus Chassis                               87
                                     Incomplete - Chassis Cab (Number of Cab Unknown)      202
                                     Incomplete - Chassis Cab (Single Cab)                 308
                                     Incomplete - Commercial Bus Chassis                    47
                                     Incomplete - Commercial Chassis                         6
                                     Incomplete - Cutaway                                  762
                                     Incomplete - Motor Home Chassis                         3
                                     Incomplete - School Bus Chassis                        22
                                     Incomplete - Shuttle Bus Chassis                        1
                                     Incomplete - Stripped Chassis                          25
                                     Minivan                                                 3
                                     Sedan/Saloon                                           10
                                     Sport Utility Vehicle (SUV)/Multi Purpose Vehic...      1
                                     Truck                                                 952
                                     Truck - Tractor                                        10
                                     Van                                                   259
                                     Wagon                                                   8
LOW SPEED VEHICLE (LSV)                                                                      1
MOTORCYCLE                                                                                  59
                                     Motorcycle - All Terrain Cycle (ATV)                    1
                                     Motorcycle - Cruiser                                  107
                                     Motorcycle - Custom                                   160
                                     Motorcycle - Dirt Bike / Off-Road                       2
                                     Motorcycle - Dual Sport / Adventure / Supermoto...     67
                                     Motorcycle - Enduro (Off-road long distance rac...     10
                                     Motorcycle - Moped                                      1
      

In [86]:
# DMV data has vehicle type variable: vehbdyt_id
# vehbdyt_id = 5 is "Suburban" 
# vehbdyt_id = 6 is "Sedan" 

veh_merge.groupby(['VEHBDYT_ID','VehicleType'])[['VIN']].count()

VIN
VEHBDYT_ID VehicleType                                
1.0        BUS                                       1
           MULTIPURPOSE PASSENGER VEHICLE (MPV)      2
           PASSENGER CAR                            19
2.0        INCOMPLETE VEHICLE                        2
3.0                                                  2
           INCOMPLETE VEHICLE                        4
           MOTORCYCLE                                1
           PASSENGER CAR                           478
4.0                                                  2
           MULTIPURPOSE PASSENGER VEHICLE (MPV)      6
           PASSENGER CAR                            44
           TRAILER                                   1
           TRUCK                                     1
5.0                                                 10
           BUS                                     597
           INCOMPLETE VEHICLE                      630
           MULTIPURPOSE PASSENGER VEHICLE (MPV)  25244
           PASSENGER CAR                           944
           TRUCK                                   815
6.0                                                 27
           INCOMPLETE VEHICLE                       17
           MOTORCYCLE                                1
           MULTIPURPOSE PASSENGER VEHICLE (MPV)    553
           PASSENGER CAR                         38327
           TRUCK                                     9
7.0                                                 14
           INCOMPLETE VEHICLE                        8
           LOW SPEED VEHICLE (LSV)                   1
           MULTIPURPOSE PASSENGER VEHICLE (MPV)     16
           PASSENGER CAR                          3126
           TRAILER                                   2
           TRUCK                                     7
8.0        INCOMPLETE VEHICLE                        6
9.0        MOTORCYCLE                                1
10.0                                                35
           MOTORCYCLE                             1704
           PASSENGER CAR                             1
           TRUCK                                     2
13.0                                                11
           INCOMPLETE VEHICLE                        1
           MOTORCYCLE                                1
15.0                                                 1
           INCOMPLETE VEHICLE                        5
           TRUCK                                     5
17.0                                                 2
           TRAILER                                   4
           TRUCK                                     1
22.0       INCOMPLETE VEHICLE                      114
23.0                                                 7
24.0                                                 1
           INCOMPLETE VEHICLE                        1
25.0       INCOMPLETE VEHICLE                        1
26.0                                                26
27.0       INCOMPLETE VEHICLE                        2
32.0       INCOMPLETE VEHICLE                        1
33.0                                                 1
           INCOMPLETE VEHICLE                        2
34.0                                                 2
           INCOMPLETE VEHICLE                        2
38.0       MOTORCYCLE                                3
40.0                                                12
           INCOMPLETE VEHICLE                       15
           TRAILER                                   3
           TRUCK                                   185
41.0       INCOMPLETE VEHICLE                      358
           MULTIPURPOSE PASSENGER VEHICLE (MPV)      1
           TRUCK                                    12
42.0       INCOMPLETE VEHICLE                      324
           TRUCK                                    10
43.0       INCOMPLETE VEHICLE                       86
           TRUCK                                     3
44.0                                

In [92]:
veh_merge.groupby(['VEHBDYT_ID','BodyClass'])[['VIN']].count()

VIN
VEHBDYT_ID BodyClass                                                
1.0        Sedan/Saloon                                           19
           Sport Utility Vehicle (SUV)/Multi Purpose Vehic...      2
           Wagon                                                   1
2.0        Incomplete - Chassis Cab (Single Cab)                   2
3.0                                                                6
           Cabriolet/Convertible                                 460
           Cargo Van                                               1
           Coupe                                                   7
           Incomplete                                              1
           Roadster                                                9
           Sedan/Saloon                                            1
4.0                                                                4
           Cabriolet/Convertible                                   1
           Coupe                                                  21
           Hatchback/Liftback/Notchback                            2
           Sedan/Saloon                                           19
           Sport Utility Vehicle (SUV)/Multi Purpose Vehic...      3
           Van                                                     1
           Wagon                                                   3
5.0                                                              198
           Bus                                                     2
           Bus - School Bus                                        1
           Cabriolet/Convertible                                   2
           Cargo Van                                             105
           Coupe                                                   2
           Crossover Utility Vehicle (CUV)                      1190
           Hatchback/Liftback/Notchback                          373
           Incomplete                                              3
           Incomplete - Chassis Cab (Number of Cab Unknown)       24
           Incomplete - Chassis Cab (Single Cab)                   8
           Incomplete - Cutaway                                  394
           Incomplete - School Bus Chassis                         2
           Incomplete - Stripped Chassis                           2
           Minivan                                              1565
           Pickup                                                263
           Sedan/Saloon                                           39
           Sedan/Saloon, Hatchback/Liftback/Notchback              1
           Sedan/Saloon, Wagon                                     9
           Sport Utility Vehicle (SUV)/Multi Purpose Vehic...  12719
           Sport Utility Vehicle (SUV)/Multi Purpose Vehic...     46
           Truck                                                   2
           Truck, Pickup                                           1
           Van                                                  3160
           Wagon                                                8112
           Wagon, Cabriolet/Convertible                            1
           Wagon, Incomplete - Chassis Cab (Number of Cab ...      2
           Wagon, Sport Utility Vehicle (SUV)/Multi Purpos...     14
6.0                                                              163
           Cabriolet/Convertible                                   5
           Cargo Van                                               1
           Coupe                                                 134
           Crossover Utility Vehicle (CUV)                         3
           Hatchback/Liftback/Notchback                         1041
           Incomplete - Chassis Cab (Number of Cab Unknown)        1
           Incomplete - Commercial Chassis                         2
           Minivan                                                26
           Pickup                                              

VEHBDYT_ID = 5 ("Suburban") matches primarily to SUV's, but almost as many are Vans and Wagons.

There may be some utility in using the VIN decoded Body Class instead of DMV's designation.

In [93]:
veh_merge.groupby(['CV_VEHMAKE_DESCR','Make']).count()[['CV_ID']]

CV_ID
CV_VEHMAKE_DESCR Make                                                     
-3                                                                       3
                 DAIMLERCHRYSLER COMMERCIAL BUS                          1
                 FORD                                                    1
                 FREIGHTLINER                                            8
                 GREAT DANE TRAILERS                                     1
                 HINO                                                    1
                 INTERNATIONAL                                           4
                 KENWORTH                                                3
                 MACK                                                    3
                 MERCURY                                                 1
                 MOTOR COACH INDUSTRIES                                  2
                 NISSAN                                                  2
                 PETERBILT                                               2
                 STERLING TRUCK                                          1
                 UD                                                      1
                 VOLVO TRUCK                                             3
ACE              TAIZHOU CHUANL MOTORCYCLE MANUFACTURING                 1
ACUR             ACURA                                                   1
ACURA                                                                    2
                 ACURA                                                1639
ADV                                                                      1
AL/RO            ALFA ROMEO                                              2
ALEXA            ALEXANDER DENNIS LIMITED                                1
ALEXD            ALEXANDER DENNIS LIMITED                                1
AM/IR            AMERICAN IRONHORSE MOTORCYCLES                          1
AM/TR            INTERNATIONAL                                           1
AMERI            FORD                                                    1
                 INTERNATIONAL                                           1
                 JEEP                                                    2
AMTRA            INTERNATIONAL                                           3
                 NAVISTAR, CATERPILLAR, IC, WORKHORSE, INTERNATI...      1
APRIL            APRILIA SPA                                             8
AS/MA            ASTON MARTIN                                            1
AUDI             AUDI                                                  594
AUTOC            AUTOCAR                                                 1
BENSO                                                                    2
BENTL                                                                    1
                 BENTLEY                                                 3
BI/BO                                                                    1
BL/BD            GMC                                                     3
BL/BI            BLUE BIRD                                              18
                 CHEVROLET                                               2
                 FORD                                                    1
                 FREIGHTLINER                                            1
                 GMC                                                     5
BL/BR            GMC                                                     1
BLBRD            GMC                                                     3
BLUEB            BLUE BIRD                                               1
                 FORD                                                    1
                 GMC                                                     8
BMW                                                                      3
                 BMW                                                  1927
BRIND              

In [96]:
veh_merge.fillna('-').groupby(['VEHBDYT_ID','BodyClass'])[['VIN']].count()\
.sort_values('VIN')

,,VIN
VEHBDYT_ID,BodyClass,
34.0,Truck,1
32.0,Truck,1
33.0,,1
6.0,"Sedan/Saloon, Wagon",1
50.0,,1
34.0,Incomplete - Chassis Cab (Single Cab),1
6.0,Incomplete - Chassis Cab (Number of Cab Unknown),1
38.0,Motorcycle - Cruiser,1
49.0,Sedan/Saloon,1
